In [9]:
# bob.refresh {"cron": "5,35 2-23 * * *", "timeout": 40000}

In [2]:
import bob
hc = bob.HealthCheck()
hc_key = "608ba515-2285-45ad-92d4-47491493ac12"
start = hc.start(hc_key)
print(start)

HealthCheck init in prod mode
Start ==> send to https://health.cashstory.com/ping/608ba515-2285-45ad-92d4-47491493ac12, 2020-04-06


In [10]:
!pip install --upgrade google-api-python-client oauth2client

Requirement already up-to-date: google-api-python-client in /opt/conda/lib/python3.7/site-packages (1.8.0)
Requirement already up-to-date: oauth2client in /opt/conda/lib/python3.7/site-packages (4.1.3)


In [11]:
import bob
import json

#-- INIT : Declare local variable
#rep
input_folder = "../data_input/"
output_folder = "../data_output/"
param_folder = "../../_parameters/"

#-- STEP0 : Connect to mongodb
#Get credentials info
credentials = param_folder + 'mongodb.json'
with open(credentials) as f:
    dbCon = json.load(f)

mongo_host = dbCon["host"]
mongo_port = dbCon["port"]
mongo_username = dbCon["username"]
mongo_password = dbCon["password"]

#Connect to mongo
MongoDB = bob.mongo.connect(mongo_host,mongo_port,mongo_username,mongo_password)

db= 'app-wsr-v1'

Successfully connected to MongoDB


In [12]:
"""Hello Analytics Reporting API V4."""

from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = param_folder + 'analytics_id_read.json'
VIEW_ID = '214312751'
METRICS = [{'expression': 'ga:users'}]
# metrics = [{'expression': 'ga:sessions'}]
DIMENTIONS = [{'name': 'ga:country'}]
FIRSTDATA = 30

def initialize_analyticsreporting():
  """Initializes an Analytics Reporting API V4 service object.

  Returns:
    An authorized Analytics Reporting API V4 service object.
  """
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)

  # Build the service object.
  analytics = build('analyticsreporting', 'v4', credentials=credentials)

  return analytics

analytics = initialize_analyticsreporting()

In [13]:
import pandas as pd
from datetime import datetime, timedelta, date
from dateutil.rrule import rrule, DAILY
from time import sleep
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import time

now = datetime.now().strftime('%d/%m/%Y %H:%M:%S')

def convert_reponse_to_df(response, date=None):
      list = []
      # parse report data
      for report in response.get('reports', []):

        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])

        for row in rows:
            dict = {}
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])

            for header, dimension in zip(dimensionHeaders, dimensions):
                dict[header] = dimension

            for i, values in enumerate(dateRangeValues):
                for metric, value in zip(metricHeaders, values.get('values')):
                    if ',' in value or ',' in value:
                        dict[metric.get('name')] = float(value)
                    else:
                        dict[metric.get('name')] = int(value)
                    if date is not None:
                        dict['date'] = date
            list.append(dict)

        df = pd.DataFrame(list)
        return df


def get_report(analytics, start_date, end_date, view_id, metrics, dimensions, dimensionFilterClauses=[], segments=[]):
    body = {
        'reportRequests': [
        {
          'viewId': view_id,
          'dateRanges': [{'startDate':start_date, 'endDate': end_date}],
          'metrics': metrics,
          'dimensions': dimensions,
          'pageSize': 10000,
          'dimensionFilterClauses': dimensionFilterClauses,
          'segments': segments,
        }]
      }
#     print('body', body)
    return analytics.reports().batchGet(
      body=body
    ).execute()


def return_ga_data(start_date, end_date, view_id, metrics, dimensions, split_dates, group_by=[], dimensionFilterClauses=[], segments=[]):
    if split_dates == False:
        return convert_reponse_to_df(get_report(analytics, start_date, end_date, view_id, metrics, dimensions, dimensionFilterClauses, segments))
    else:
        start_date = datetime.strptime(start_date, '%Y-%m-%d').date()
        end_date = datetime.strptime(end_date, '%Y-%m-%d').date()

        df_total = pd.DataFrame()
        for date in rrule(freq=DAILY, dtstart=start_date, until=end_date):
            date = str(date.date())
            df_total = df_total.append(convert_reponse_to_df(get_report(analytics, date, date, view_id, metrics, dimensions, dimensionFilterClauses, segments), date))

        if len(group_by) != 0:
            df_total = df_total.groupby(group_by).sum()

        return df_total

thirtydaysAgo = (datetime.now() - timedelta(FIRSTDATA)).strftime("%Y-%m-%d")
today = datetime.now().strftime("%Y-%m-%d")

start_time = time.time()
df = return_ga_data(thirtydaysAgo, today, VIEW_ID, METRICS, DIMENTIONS, True)
print("Script execution completed at " + now + ". Time: --- %s secnds ---" % (time.time() - start_time))

Script execution completed at 04/04/2020 11:04:57. Time: --- 4.897118806838989 secnds ---


In [14]:
#Save traffic
df['LAST_UPDATE'] = now
bob.mongo.save_df(df,'DB_TRAFIC',db,False)

Dataframe DB_TRAFIC successfully save in database app-wsr-v1 in MongoDB. Time: --- 0.015610933303833008 secnds ---


In [15]:
def domain_301(df):
    cols_to_rename = {"ga:country": "ENTITY", "date": "DATE", "ga:users": "VALUE"}
    domain = df.copy().rename(index=str, columns=cols_to_rename)
    domain['DATE_ORDER'] = pd.to_datetime(domain['DATE'], format='%Y-%m-%d').dt.strftime('%Y%m%d')
    
    #-- Consolidate data
    domain_ww = domain.copy()
    domain_ww['ENTITY'] = 'WORLDWIDE'
    
    # Concat & groupby
    domain = pd.concat([domain,domain_ww], axis=0)
    domain = domain.groupby(['ENTITY','DATE','DATE_ORDER'], as_index=False).agg({'VALUE': 'sum',})
    
    domain['LAST_UPDATE'] = now
    
    return domain

domain301 = domain_301(df)
bob.mongo.save_df(domain301,'301',db,True)

Dataframe 301 successfully save in database app-wsr-v1 in MongoDB. Time: --- 0.030979156494140625 secnds ---


In [16]:
domain301 = domain301[domain301['ENTITY'] == 'WORLDWIDE'].sort_values("DATE_ORDER",ascending=False)
domain301.head(5)

,ENTITY,DATE,DATE_ORDER,VALUE,LAST_UPDATE
428,WORLDWIDE,2020-04-04,20200404,135,04/04/2020 11:04:57
427,WORLDWIDE,2020-04-03,20200403,698,04/04/2020 11:04:57
426,WORLDWIDE,2020-04-02,20200402,815,04/04/2020 11:04:57
425,WORLDWIDE,2020-04-01,20200401,1221,04/04/2020 11:04:57
424,WORLDWIDE,2020-03-31,20200331,1073,04/04/2020 11:04:57


In [3]:
done = hc.done(hc_key)
print(done)

End ==> send to https://health.cashstory.com/ping/608ba515-2285-45ad-92d4-47491493ac12, 2020-04-06
